In [5]:
#トラカンデータの距離案分()
import numpy as np
import psycopg2
import datetime
import pandas as pd

In [38]:
#計測地点のosm_idを指定
start_osm = 51396950
mid_osm = [211152185]
end_osm =  211152194
start_mesh = 50325632424
end_mesh = 50325623313

In [39]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [40]:
getdbname = "kddi_00_new_param"
dbname = "kddi_trkn00_monthly_anbun"
trknName = "trkn_00_grp"
sttime = "2022-04-01 00:00:00"

In [41]:
for i in range(30):
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    if i != 0:
        dte = dte + datetime.timedelta(days=i)
    for j in range(24):
        timestr = dte.strftime('%H:%M:%S')
        #特定日付のKDDIメッシュ情報を全取得
        cur =connection.cursor()
        sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM " + getdbname + " "
        sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
        sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
        cur.execute(sql)
        kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
        cur.close()

        one_target_list = []
        one_target_list.append(start_mesh)
        one_target_list.append(end_mesh)
        two_target_list = []
        two_target_list.append(start_osm)
        if len(mid_osm) != 0:
            for target in mid_osm:
                two_target_list.append(target)
        if start_osm != end_osm:
            two_target_list.append(end_osm)

        cur =connection.cursor()
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            stlat = row[0]
            stlng = row[1]
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            edlat = row[0]
            edlng = row[1]
        cur.close()

        if stlat > edlat:
            wklat = stlat
            stlat = edlat
            edlat = wklat
        if stlng > edlng:
            wklng = stlng
            stlng = edlng
            edlng = wklng

        print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))

        keylist = []
        cnt = 0
        for item in two_target_list:
            sql = "SELECT a.grid_code,a.latitude,a.longitude"
            sql += " FROM pointdata a, roadmap r"
            sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
            sql += " order by ST_Distance(a.geom, r.geom);"
            cur =connection.cursor()
            cur.execute(sql)
            childlist = []
            for row in cur:
                if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                    childlist.append([row[0], row[1], row[2]])
                #print(row[0])
            worklist = [item, childlist]
            keylist.append(worklist)
            cnt = cnt + 1
            cur.close()
        #特定日付のトラカンデータを全取得 > 緯度経度
        cur =connection.cursor()
        #sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
        #sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
        #sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
        #sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
        #sql += "ORDER BY trakanmap.gridcode"
        sql = "SELECT grid_code, tbran, latitude, longitude FROM " + trknName  + " "
        sql += "INNER JOIN pointdata ON mesh = cast(grid_code as bigint) "
        sql += "WHERE times = '"+timestr+"' AND (mesh = '"+str(start_mesh)+"' or mesh = '"+str(end_mesh)+"') "
        sql += "ORDER BY mesh"
        #print(sql)
        cur.execute(sql)
        trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
        cur.close()
        trknDf
        #keylist

        from geopy.distance import geodesic

        #データが存在する道路IDごとに交通量を平均化して取得する
        lstItems = []
        cntDt = 0
        wks = ""
        wknp=""
        find = 0

        trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
        trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')
        trknStParam

        if len(trknStParam) == 0:
            print(str(start_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknStParam) == 1:
            trknSt = trknStParam['danmen'].iloc[-1]
        else:
            trknSt = trknStParam['danmen'].iloc[0] + trknStParam['danmen'].iloc[1]

        if len(trknEdParam) == 0:
            print(str(end_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknEdParam) == 1:
            trknEd = trknEdParam['danmen'].iloc[-1]
        else:
            trknEd = trknEdParam['danmen'].iloc[0] + trknEdParam['danmen'].iloc[1]

        st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
        ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]

        dis_km = geodesic(st_station, ed_station)
        cur =connection.cursor()
        for row in keylist:
            for meshid in row[1]:
                #路線のメッシュ数分ループ
                print(str(meshid[0]))
                kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                if len(kddiParam) == 0:
                    print(str(meshid[0]) + " is kddi none!")
                    continue
                #路線のメッシュがkddiに存在したなら
                
                kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                kddi_km = geodesic(st_station, kddi_station)
                
                if trknSt < trknEd:
                    ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                else:
                    ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                cur.execute(sql)
                connection.commit()

        sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
        cur.execute(sql)
        connection.commit()
        cur.close()
        dte = dte + datetime.timedelta(minutes=15) 
        #print(lstItems)


33.7723958335:33.7807291665:132.786718750:132.786718750
50325632422
存在メッシュ：50325632422値1.7959193507087008距離0.11554000478983723 km
50325632424
50325632424 is kddi none!
50325633313
存在メッシュ：50325633313値1.9567582970135229距離0.14472107054004557 km
50325633311
50325633311 is kddi none!
50325632422
存在メッシュ：50325632422値1.7959193507087008距離0.11554000478983723 km
50325632244
存在メッシュ：50325632244値2.4327476846567997距離0.23107999004512475 km
50325633311
50325633311 is kddi none!
50325632242
存在メッシュ：50325632242値3.069576216599194距離0.346620011222586 km
50325632244
存在メッシュ：50325632244値2.4327476846567997距離0.23107999004512475 km
50325632224
存在メッシュ：50325632224値3.7064043351871323距離0.462159957405005 km
50325623313
存在メッシュ：50325623313値6.315789473684211距離0.9355816070954379 km
50325632222
存在メッシュ：50325632222値4.343232346100218距離0.5776998840519383 km
50325622444
存在メッシュ：50325622444値4.980060555025782距離0.6932398466243439 km
50325623331
存在メッシュ：50325623331値5.687698333553741距離0.821626813941455 km
50325622442
50325622442 is kdd

33.7723958335:33.7807291665:132.786718750:132.786718750
50325632422
50325632422 is kddi none!
50325632424
50325632424 is kddi none!
50325633313
50325633313 is kddi none!
50325633311
50325633311 is kddi none!
50325632422
50325632422 is kddi none!
50325632244
存在メッシュ：50325632244値1.532001473534355距離0.23107999004512475 km
50325633311
50325633311 is kddi none!
50325632242
存在メッシュ：50325632242値1.9408594457122237距離0.346620011222586 km
50325632244
存在メッシュ：50325632244値1.532001473534355距離0.23107999004512475 km
50325632224
存在メッシュ：50325632224値2.349717152507408距離0.462159957405005 km
50325623313
存在メッシュ：50325623313値4.025距離0.9355816070954379 km
50325632222
存在メッシュ：50325632222値2.7585747901729616距離0.5776998840519383 km
50325622444
存在メッシュ：50325622444値3.167432554966896距離0.6932398466243439 km
50325623331
50325623331 is kddi none!
50325622442
50325622442 is kddi none!
50325623333
存在メッシュ：50325623333値3.2203215148453532距離0.7081858473391164 km
50325633111
存在メッシュ：50325633111値2.8217488012591736距離0.5955523586897342 km


33.7723958335:33.7807291665:132.786718750:132.786718750
50325632422
50325632422 is kddi none!
50325632424
50325632424 is kddi none!
50325633313
存在メッシュ：50325633313値0.9654750364987309距離0.14472107054004557 km
50325633311
50325633311 is kddi none!
50325632422
50325632422 is kddi none!
50325632244
50325632244 is kddi none!
50325633311
50325633311 is kddi none!
50325632242
50325632242 is kddi none!
50325632244
50325632244 is kddi none!
50325632224
存在メッシュ：50325632224値1.9316363892042374距離0.462159957405005 km
50325623313
存在メッシュ：50325623313値3.372549019607843距離0.9355816070954379 km
50325632222
存在メッシュ：50325632222値2.2832952956575743距離0.5776998840519383 km
50325622444
存在メッシュ：50325622444値2.634954311454172距離0.6932398466243439 km
50325623331
存在メッシュ：50325623331値3.0257146472085816距離0.821626813941455 km
50325622442
50325622442 is kddi none!
50325623333
存在メッシュ：50325623333値2.6804441643538417距離0.7081858473391164 km
50325633111
50325633111 is kddi none!
50325622424
50325622424 is kddi none!
50325633113
503256

33.7723958335:33.7807291665:132.786718750:132.786718750
50325632422
存在メッシュ：50325632422値1.9046893612813158距離0.11554000478983723 km
50325632424
50325632424 is kddi none!
50325633313
存在メッシュ：50325633313値2.034135820679589距離0.14472107054004557 km
50325633311
50325633311 is kddi none!
50325632422
存在メッシュ：50325632422値1.9046893612813158距離0.11554000478983723 km
50325632244
存在メッシュ：50325632244値2.4172217731627725距離0.23107999004512475 km
50325633311
50325633311 is kddi none!
50325632242
存在メッシュ：50325632242値2.92975434439407距離0.346620011222586 km
50325632244
存在メッシュ：50325632244値2.4172217731627725距離0.23107999004512475 km
50325632224
存在メッシュ：50325632224値3.4422865829492837距離0.462159957405005 km
50325623313
50325623313 is kddi none!
50325632222
50325632222 is kddi none!
50325622444
50325622444 is kddi none!
50325623331
存在メッシュ：50325623331値5.036872340987891距離0.821626813941455 km
50325622442
50325622442 is kddi none!
50325623333
存在メッシュ：50325623333値4.5336511220272495距離0.7081858473391164 km
50325633111
存在メッシュ：5032

33.7723958335:33.7807291665:132.786718750:132.786718750
50325632422
存在メッシュ：50325632422値1.3431546232921665距離0.11554000478983723 km
50325632424
存在メッシュ：50325632424値0.8490566037735849距離0.0 km
50325633313
50325633313 is kddi none!
50325633311
50325633311 is kddi none!
50325632422
存在メッシュ：50325632422値1.3431546232921665距離0.11554000478983723 km
50325632244
存在メッシュ：50325632244値1.837252559272736距離0.23107999004512475 km
50325633311
50325633311 is kddi none!
50325632242
50325632242 is kddi none!
50325632244
存在メッシュ：50325632244値1.837252559272736距離0.23107999004512475 km
50325632224
存在メッシュ：50325632224値2.8254484177601684距離0.462159957405005 km
50325623313
50325623313 is kddi none!
50325632222
存在メッシュ：50325632222値3.3195461031065805距離0.5776998840519383 km
50325622444
存在メッシュ：50325622444値3.8136439420855357距離0.6932398466243439 km
50325623331
存在メッシュ：50325623331値4.362681017155496距離0.821626813941455 km
50325622442
50325622442 is kddi none!
50325623333
存在メッシュ：50325623333値3.8775593743752004距離0.7081858473391164 km
50

...

33.7723958335:33.7807291665:132.786718750:132.786718750
50325632422
存在メッシュ：50325632422値1.155628977155685距離0.11554000478983723 km
50325632424
存在メッシュ：50325632424値0.7804878048780488距離0.0 km
50325633313
存在メッシュ：50325633313値1.2503755476942282距離0.14472107054004557 km
50325633311
50325633311 is kddi none!
50325632422
存在メッシュ：50325632422値1.155628977155685距離0.11554000478983723 km
50325632244
存在メッシュ：50325632244値1.5307700860075504距離0.23107999004512475 km
50325633311
50325633311 is kddi none!
50325632242
50325632242 is kddi none!
50325632244
存在メッシュ：50325632244値1.5307700860075504距離0.23107999004512475 km
50325632224
50325632224 is kddi none!
50325623313
50325623313 is kddi none!
50325632222
存在メッシュ：50325632222値2.65619321204072距離0.5776998840519383 km
50325622444
50325622444 is kddi none!
50325623331
50325623331 is kddi none!
50325622442
50325622442 is kddi none!
50325623333
存在メッシュ：50325623333値3.0798616836005746距離0.7081858473391164 km
50325633111
存在メッシュ：50325633111値2.7141575359635164距離0.5955523586897342 

33.7723958335:33.7807291665:132.786718750:132.786718750
50325632422
存在メッシュ：50325632422値1.336880692775802距離0.11554000478983723 km
50325632424
存在メッシュ：50325632424値0.9795918367346939距離0.0 km
50325633313
存在メッシュ：50325633313値1.427118438731559距離0.14472107054004557 km
50325633311
50325633311 is kddi none!
50325632422
存在メッシュ：50325632422値1.336880692775802距離0.11554000478983723 km
50325632244
50325632244 is kddi none!
50325633311
50325633311 is kddi none!
50325632242
存在メッシュ：50325632242値2.051458395126657距離0.346620011222586 km
50325632244
50325632244 is kddi none!
50325632224
50325632224 is kddi none!
50325623313
50325623313 is kddi none!
50325632222
存在メッシュ：50325632222値2.7660356843305265距離0.5776998840519383 km
50325622444
存在メッシュ：50325622444値3.123324409821027距離0.6932398466243439 km
50325623331
50325623331 is kddi none!
50325622442
50325622442 is kddi none!
50325623333
存在メッシュ：50325623333値3.169542509841742距離0.7081858473391164 km
50325633111
存在メッシュ：50325633111値2.8212415868440606距離0.5955523586897342 km
50

33.7723958335:33.7807291665:132.786718750:132.786718750
50325632422
存在メッシュ：50325632422値2.2995566282444453距離0.11554000478983723 km
50325632424
存在メッシュ：50325632424値1.6181818181818182距離0.0 km
50325633313
存在メッシュ：50325633313値2.471646314142763距離0.14472107054004557 km
50325633311
50325633311 is kddi none!
50325632422
存在メッシュ：50325632422値2.2995566282444453距離0.11554000478983723 km
50325632244
存在メッシュ：50325632244値2.980931323105847距離0.23107999004512475 km
50325633311
50325633311 is kddi none!
50325632242
存在メッシュ：50325632242値3.662306229811314距離0.346620011222586 km
50325632244
存在メッシュ：50325632244値2.980931323105847距離0.23107999004512475 km
50325632224
存在メッシュ：50325632224値4.343680694248038距離0.462159957405005 km
50325623313
存在メッシュ：50325623313値7.135593220338983距離0.9355816070954379 km
50325632222
存在メッシュ：50325632222値5.025055043478018距離0.5776998840519383 km
50325622444
存在メッシュ：50325622444値5.706429604571515距離0.6932398466243439 km
50325623331
存在メッシュ：50325623331値6.463566889006766距離0.821626813941455 km
50325622442
50

50325632422
50325632422 is kddi none!
50325632424
50325632424 is kddi none!
50325633313
50325633313 is kddi none!
50325633311
50325633311 is kddi none!
50325632422
50325632422 is kddi none!
50325632244
50325632244 is kddi none!
50325633311
50325633311 is kddi none!
50325632242
50325632242 is kddi none!
50325632244
50325632244 is kddi none!
50325632224
50325632224 is kddi none!
50325623313
50325623313 is kddi none!
50325632222
50325632222 is kddi none!
50325622444
50325622444 is kddi none!
50325623331
50325623331 is kddi none!
50325622442
50325622442 is kddi none!
50325623333
50325623333 is kddi none!
50325633111
50325633111 is kddi none!
50325622424
50325622424 is kddi none!
50325633113
50325633113 is kddi none!
50325633133
50325633133 is kddi none!
50325633131
50325633131 is kddi none!
50325632422
50325632422 is kddi none!
33.7723958335:33.7807291665:132.786718750:132.786718750
50325632422
50325632422 is kddi none!
50325632424
50325632424 is kddi none!
50325633313
50325633313 is kddi 

50325632422
50325632422 is kddi none!
50325632424
50325632424 is kddi none!
50325633313
50325633313 is kddi none!
50325633311
50325633311 is kddi none!
50325632422
50325632422 is kddi none!
50325632244
50325632244 is kddi none!
50325633311
50325633311 is kddi none!
50325632242
50325632242 is kddi none!
50325632244
50325632244 is kddi none!
50325632224
50325632224 is kddi none!
50325623313
50325623313 is kddi none!
50325632222
50325632222 is kddi none!
50325622444
50325622444 is kddi none!
50325623331
50325623331 is kddi none!
50325622442
50325622442 is kddi none!
50325623333
50325623333 is kddi none!
50325633111
50325633111 is kddi none!
50325622424
50325622424 is kddi none!
50325633113
50325633113 is kddi none!
50325633133
50325633133 is kddi none!
50325633131
50325633131 is kddi none!
50325632422
50325632422 is kddi none!
33.7723958335:33.7807291665:132.786718750:132.786718750
50325632422
50325632422 is kddi none!
50325632424
50325632424 is kddi none!
50325633313
50325633313 is kddi 

In [42]:
connection.close()

In [27]:
#CreateGeojsonFile